# Pre-Processing of Slocum Glider-AD2CP Data: RU29 2020

jgradone@marine.rutgers.edu     06/26/2022    Initial

**This Jupyter Notebook is intended to:**<br>
1) Read in AD2CP data processed to NetCDFs using the Nortek MIDAS software as an xarray DataSet <br>
2) Correct for local speed of sound <br>
3) QAQC AD2CP data <br>
4) Correct beams to level true-depth <br>
5) Bin map AD2CP data <br>
6) AD2CP Coordinate transformation from beam to XYZ to ENU <br>
7) QAQC AD2CP data post coordinate transformation <br>
10) Save output from each file <br>

**It is strategic to do this on the individual NetCDFs. Once this is done an entire deployment, a cleaned and processed dataset is created that can be used for inversion/shear method testing much faster.**

*Details/comments on what the functions are actually doing in the source code*

In [1]:
# Imports
import scipy.interpolate as interp
from scipy.sparse.linalg import lsqr
import scipy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import netCDF4 as nc
import math
import datetime
import xarray as xr
import matplotlib.dates as mdates
import dask.array as da
from erddapy import ERDDAP
from netCDF4 import Dataset
import gsw
import cmocean.cm as cmo
import sys
from datetime import datetime


## To import functions from Slocum-AD2CP GitHub repository, make this path the path to where the repo exists locally
sys.path.insert(0,'/home/jg1200/GitHub/Slocum-AD2CP/src/analysis/')
sys.path.insert(0,'/home/jg1200/GitHub/Slocum-AD2CP/src/data/')
from make_dataset import correct_sound_speed, beam_true_depth, cell_vert, binmap_adcp, beam2enu, inversion, qaqc_pre_coord_transform, qaqc_post_coord_transform, shear_method
from analysis import get_erddap_dataset


## Loop to read NetCDF and perform Steps 2-10

In [ ]:
path = '/home/jg1200/Data/GliderData/RU29/RU29_2020_AD2CP_NC/'
files = np.sort(glob.glob(path+'*.nc'))

for x in np.arange(0,len(files)):

    ## Load AD2CP data
    ad2cp = xr.open_dataset(files[x],group='Data/Burst/')

    ## Load metdata
    config = xr.open_dataset(files[x],group='Config')
    ad2cp = ad2cp.assign_attrs(config.attrs)

    ## Rename a few variables so there are no spaces in names
    ad2cp = ad2cp.rename({'Velocity Range':'VelocityRange','Correlation Range':'CorrelationRange','Amplitude Range':'AmplitudeRange'})
    # 2 House-keeping steps
    # 1) Roll shifted 180 for some reason
    ad2cp['Roll'] = ad2cp['Roll']-180
    # 2) Surface depth is 10 meters. Needed for at least for the 2020 and both 2021 deployments, per conversations with Sven from Nortek.
    ad2cp['Pressure'] = ad2cp['Pressure']-10
    # Put time on x-dimension
    ad2cp = ad2cp.transpose()


    ad2cp = correct_sound_speed(ad2cp)
    ad2cp = qaqc_pre_coord_transform(ad2cp, corr_threshold = 50, max_amplitude = 75)
    ad2cp = beam_true_depth(ad2cp)
    ad2cp = binmap_adcp(ad2cp)
    ad2cp = beam2enu(ad2cp)
    ad2cp = qaqc_post_coord_transform(ad2cp, high_velocity_threshold=0.75, surface_depth_to_filter = 5)

    ## Save just a subset for size/speed efficiency
    variables = [
        'time',
        'VelocityRange',
        'UVelocity',
        'VVelocity',
        'WVelocity',
        'Heading',
        'Pressure'
    ]
    subset_ad2cp = ad2cp[variables]
    
    ## Save in scratch!
    fname = '/scratch/jg1200/Data/GliderData/RU29/RU29_2020/Pre_Processed/RU29_2020_Pre_Processed_{}.nc'.format(x)
    subset_ad2cp.to_netcdf(fname)
    print('Finished File {} out of {} at {}'.format(x,len(files),datetime.now().strftime("%m/%d/%y %H:%M:%S")))



Finished File 0 out of 92 at 06/26/22 20:21:21
Finished File 1 out of 92 at 06/26/22 20:23:38
Finished File 2 out of 92 at 06/26/22 20:26:04
Finished File 3 out of 92 at 06/26/22 20:28:33
Finished File 4 out of 92 at 06/26/22 20:30:58
Finished File 5 out of 92 at 06/26/22 20:33:25
Finished File 6 out of 92 at 06/26/22 20:35:52
Finished File 7 out of 92 at 06/26/22 20:38:16
Finished File 8 out of 92 at 06/26/22 20:40:12
Finished File 9 out of 92 at 06/26/22 20:42:05
Finished File 10 out of 92 at 06/26/22 20:44:03
Finished File 11 out of 92 at 06/26/22 20:46:02
Finished File 12 out of 92 at 06/26/22 20:48:07
Finished File 13 out of 92 at 06/26/22 20:50:11
Finished File 14 out of 92 at 06/26/22 20:52:15
Finished File 15 out of 92 at 06/26/22 20:54:21
Finished File 16 out of 92 at 06/26/22 20:56:26
Finished File 17 out of 92 at 06/26/22 20:58:32
Finished File 18 out of 92 at 06/26/22 21:00:39
Finished File 19 out of 92 at 06/26/22 21:02:46
Finished File 20 out of 92 at 06/26/22 21:04:50
Fi